In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, classification_report 
import plotly.express as px
%matplotlib inline

# Load data

In [2]:
df = pd.read_csv("/Users/irene/CloudStation/Courses/ARP/ARP code/Maserati-ARP/Maserati-ARP/Data-analysis/sport_sentiment.csv")

In [3]:
df = df.drop(columns="Unnamed: 0")

In [4]:
df[['topic number', 'Comment']]

,topic number,Comment
0,0,"Amazing!Red, the best colour for a Maserati"
1,0,The closest other Maserati colour to Rosso Vin...
2,0,Interesting that a GT3 version is supposedly a...
3,0,MC20 - 2022 winnerGranTurismo Folgore - 2023 w...
4,0,"So the essence is "" I adore this thing, this ..."
...,...,...
25785,0,This is purely an observation (or I am being b...
25786,0,What we are trying to do is make sure that all...
25787,0,"""Anything goes Robin that is member orientated..."
25788,0,Just seeing if anyone else uses that forum or ...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25790 entries, 0 to 25789
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tokens        25790 non-null  object
 1   topic number  25790 non-null  int64 
 2   Comment       25790 non-null  object
dtypes: int64(1), object(2)
memory usage: 604.6+ KB


In [5]:
df1 = df.copy()

# Sentiment analysis

# Unsupervised Lexicon Models

# Vadar comment

In [6]:
vadar_df = df1.copy()

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/irene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/irene/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
import re
def trim_length(text):
    '''
    This method removes the repeating characters that are repeated more than 2 times
    '''
    pat = re.compile(r"(.)\1{2,}")
    return pat.sub(r"\1\1", text)
 
def word_correct(myStr):
    '''
    This method accepts a string and removes the meta characters from it
    '''
    myStr = str(myStr)
    pat = re.compile(r'[^a-zA-Z1-9]+')
    corrected_str = ''
    splits = myStr.split()
    for word in splits:
        word = word.strip()
        word = re.sub(pat, '', word).lower()
        word = trim_length(word)
        corrected_str = corrected_str+word+' '
    corrected_str = corrected_str.strip()
    return corrected_str
 
def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
 
def lemmatize_word(myStr):
    myStr = str(myStr)
    lemmatizer = WordNetLemmatizer()
    lst_word = nltk.word_tokenize(myStr)
    final_str=''
    for word in lst_word:
        word = lemmatizer.lemmatize(word, get_pos(word))
        final_str = final_str+word+' '
 
    final_str = final_str.strip()
    return final_str
 
vadar_df['Comment'] = vadar_df['Comment'].apply(lambda i:word_correct(i))
vadar_df['Comment'] = vadar_df['Comment'].apply(lambda i: lemmatize_word(i))

In [9]:
vadar_df1 = vadar_df.copy()

In [11]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm

def func_intensity_analyser(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)

    positive = round(scores['pos'], 2) * 100
    negative = round(scores['neg'], 2) * 100
    neutral = round(scores['neu'], 2) * 100
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold else 'negative'

    return negative, positive, neutral, final_sentiment

vadar_df1['neg'], vadar_df1['pos'], vadar_df1['neu'], vadar_df1['sentiment'] = zip(*[func_intensity_analyser(r, threshold=0.4) for r in tqdm(vadar_df1.Comment)])

  0%|          | 5/25790 [00:00<09:36, 44.71it/s]

100%|██████████| 25790/25790 [04:12<00:00, 102.12it/s]


In [16]:
vadar_df_2 = vadar_df1[["topic number","neg","pos"]]

In [17]:
avg_scores = vadar_df_2.groupby('topic number').agg({'neg': 'mean', 'pos': 'mean'}).reset_index()

In [21]:
avg_scores["neg"] = round(avg_scores["neg"],2)

In [23]:
avg_scores["pos"] = round(avg_scores["pos"],2)

In [24]:
avg_scores

,topic number,neg,pos
0,0,4.23,11.35
1,1,7.00,3.50
2,2,0.97,1.97
3,3,5.48,7.79
4,4,2.42,4.68
5,5,4.78,6.93


In [25]:
avg_scores['topic sentiment'] = avg_scores.apply(lambda row: 'positive' if row['pos'] > row['neg']
                                           else 'negative' if row['pos'] < row['neg']
                                           else 'neutral', axis=1)

In [27]:
avg_scores

,topic number,neg,pos,topic sentiment
0,0,4.23,11.35,positive
1,1,7.00,3.50,negative
2,2,0.97,1.97,positive
3,3,5.48,7.79,positive
4,4,2.42,4.68,positive
5,5,4.78,6.93,positive
